# <center>CryptoCurrencies CA#2<center>
<h1><center>Transacting on Bitcoin TestNet / Firing up an Ethereum Node<center></h1>

#### <center>"Name - Family name"<center>
#### <center>"Student ID"<center>

### 1. Address Generation:

 1.1: Base58 and WIF address

In [1]:
from bit import PrivateKeyTestnet


def generate_wif():
    return PrivateKeyTestnet().to_wif()

def generate_testnet_address(wif):
    return PrivateKeyTestnet(wif)

wif = generate_wif()

my_key = generate_testnet_address(wif)
print(my_key.version)
print(my_key.to_wif())
print(my_key.address)


test
cVJv9pWzLRizhfRFNa2y7YsLvdc2sVdwFEfgMenCDF6LYcaX7atg
mjYsji2z7HtGXUbRquR7kU3WhAEDBZF7op


In [20]:
import os
import hashlib
from coincurve import PublicKey
from bech32 import bech32_encode, convertbits

# Generate a new private key
private_key = os.urandom(32)

# Generate the public key
public_key = PublicKey.from_valid_secret(private_key).format(compressed=True)

# Generate the public key hash
public_key_hash = hashlib.new('ripemd160', hashlib.sha256(public_key).digest()).digest()

# Convert the public key hash to base32
public_key_hash_base32 = convertbits(public_key_hash, 8, 5)

# Generate the Bech32 address
address = bech32_encode('tb', public_key_hash_base32)

print("Private Key: ", private_key.hex())
print("Bitcoin Address: ", address)


Private Key:  fbd2834817a62873059f2d6be5c57a57811a39087bbb51732c223a194c97fca4
Bitcoin Address:  tb1qtgwypa4fk4jkja0acvpwmc2hq8nms6pw8vkms


In [3]:
import os
import hashlib
import bech32
import ecdsa
import base58

def hash160(s):
    return hashlib.new('ripemd160', hashlib.sha256(s).digest()).digest()

def pubkey_to_address(pubkey: bytes):
    sha = hashlib.sha256(pubkey).digest()
    ripemd = hash160(sha)
    data = bech32.convertbits(ripemd, 8, 5)
    bech32_addr = bech32.bech32_encode('tb', [0] + data)

    return bech32_addr

# Generate a new raw private key
raw_private_key = os.urandom(32)

# Add a 0x80 byte in front of it for mainnet addresses or 0xef for testnet addresses
# Here we choose 0xef for a testnet address
extended_key = b'\xef' + raw_private_key

# Add a 0x01 byte at the end if the private key will correspond to a compressed public key
extended_key += b'\x01'

# Perform SHA-256 hash on the extended key
hash = hashlib.sha256(extended_key).digest()

# Perform SHA-256 hash on the result of the previous SHA-256 hash
hash = hashlib.sha256(hash).digest()

# Take the first 4 bytes of the second SHA-256 hash, this is the checksum
checksum = hash[:4]

# Add the 4 checksum bytes from stage 7 at the end of the extended key from stage 6
wif_bytes = extended_key + checksum

# Convert the result from a byte string into a base58 string using Base58Check encoding
# This is the Wallet Import Format
wif_private_key = base58.b58encode(wif_bytes).decode('utf-8')

# Create a SigningKey object from the raw private key
private_key = ecdsa.SigningKey.from_string(raw_private_key, curve=ecdsa.SECP256k1)

# Get the public key
public_key = private_key.get_verifying_key().to_string()

# Get the Bitcoin testnet address
address = pubkey_to_address(public_key)

print('WIF Private Key:', wif_private_key)
print('Bitcoin Testnet Address:', address)


WIF Private Key: cUkSiHJTo7dqpsadJGJ9FVQVCZeken8t1k3WcMgTV23DHo5ah5Kr
Bitcoin Testnet Address: tb1qzlkszsz7xe95slzqzc09wsvv535grundvxykwd


1.2: Vanity Address Generator


In [9]:
import os
import hashlib
import base58
import ecdsa
from datetime import datetime

def generate_private_key():
    return os.urandom(32)

def private_key_to_public_key(private_key):
    sk = ecdsa.SigningKey.from_string(private_key, curve=ecdsa.SECP256k1)
    vk = sk.get_verifying_key()
    return b'\x04' + vk.to_string()

def public_key_to_address(public_key):
    sha256_pk = hashlib.sha256(public_key).digest()

    ripemd160_pk = hashlib.new('ripemd160', sha256_pk).digest()

    versioned_payload = b'\x00' + ripemd160_pk

    checksum = hashlib.sha256(versioned_payload).digest()
    checksum = hashlib.sha256(checksum).digest()

    checksum = checksum[:4]

    binary_address = versioned_payload + checksum

    address = base58.b58encode(binary_address)

    return address

def generate_vanity_address(pattern):
    near_vanity_count = 0
    while True:
        private_key = generate_private_key()
        public_key = private_key_to_public_key(private_key)
        address = public_key_to_address(public_key).decode()
        if address.startswith("1Va"):
            now = datetime.now()
            near_vanity_count += 1
            print(f'{near_vanity_count} : {address} at time : {now.hour} : {now.minute}')

        if address.startswith(pattern):
            return private_key.hex(), address

desired_pattern = "1Vay"
private_key, address = generate_vanity_address(desired_pattern)
print("Private Key:", private_key)
print("Bitcoin Address:", address)


1 : 1VancbtbWExbvPEYr8yktx1fvv4S4xcPv at time : 22 : 12
2 : 1Va8ZbXX5WoEo9k1QgcpkWv5nSUtySajw at time : 22 : 13
3 : 1VaVmnP5JDpUBdL9nf4Lkgvf4oGVMtZ75 at time : 22 : 13
4 : 1VaKBHzUovyvB8uvXpY9FYxEzpUTRKQku at time : 22 : 14
5 : 1VaHUoBvXuz8MuyALd8E9TNNobTff9cTc at time : 22 : 14
6 : 1Vav3tGVhqFRobo3DrstbGUkzhUHafsJx at time : 22 : 15
7 : 1VaMFFPjhceWEokN1eKU2Uy94z5KdGK2N at time : 22 : 16
8 : 1VaMsie7TGsamDARs6gRnHW7Ax158pmFv at time : 22 : 17
9 : 1Va7DXoioVYCYf5PpyGbJVuapSa2yKfCa at time : 22 : 17
10 : 1Va4F5nC5WqcPuWqnnTY9w3ZirWim1dJE at time : 22 : 18
11 : 1VagJZPknpTnCBVc3pd8PsYDXugBiWHzP at time : 22 : 18
12 : 1VaTm1jPg1ZYPJ8QHPixowauUc5ZSBU91 at time : 22 : 25
13 : 1VayEauWuP6guh3W4n8FpEg3LHPbTt8NU at time : 22 : 28
Private Key: d0e37f7a8c201cc61776cc48ac3027c18273b931ca84f7338ebec8066cf3661d
Bitcoin Address: 1VayEauWuP6guh3W4n8FpEg3LHPbTt8NU


### 2. Transacting on Bitcoin TestNet:

Here are some utility functions that you will need to use in order to make transactions. Fill the #TODO parts in the functions in order to be used in the next steps:

#### Utility Functions:

In [ ]:
import bitcoin.wallet
from bitcoin.core import COIN, b2lx, serialize, x, lx, b2x
from utils import *

bitcoin.SelectParams("testnet")  # Select the network (testnet or mainnet)
# Private key in WIF format XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
my_private_key = bitcoin.wallet.CBitcoinSecret("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
my_public_key = my_private_key.pub
my_address = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(my_public_key)
destination_address = bitcoin.wallet.CBitcoinAddress('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')  # Destination address (recipient of the money)


def P2PKH_scriptPubKey(address):
    ######################################################################
    ## Fill out the operations for P2PKH scriptPubKey                   ##

    return [..., OP_CHECKSIG]  # Fill this section
    ######################################################################


def P2PKH_scriptSig(txin, txout, txin_scriptPubKey):
    ######################################################################
    ## Fill out the operations for P2PKH scriptSig                      ##

    signature = create_OP_CHECKSIG_signature(txin, txout, txin_scriptPubKey, my_private_key)

    return [signature, ...]  # Fill this section
    ######################################################################


def send_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_index, txout_scriptPubKey):
    txout = create_txout(amount_to_send, txout_scriptPubKey)

    txin_scriptPubKey = P2PKH_scriptPubKey(my_address)
    txin = create_txin(txid_to_spend, utxo_index)
    txin_scriptSig = P2PKH_scriptSig(txin, txout, txin_scriptPubKey)

    new_tx = create_signed_transaction(txin, txout, txin_scriptPubKey, txin_scriptSig)

    return broadcast_transaction(new_tx)


def run():
    ######################################################################
    amount_to_send = 0.1
    txid_to_spend = ('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')  # TxHash of UTXO
    utxo_index = 0  # UTXO index among transaction outputs
    ######################################################################

    print(my_address)  # Prints your address in base58
    print(my_public_key.hex())  # Print your public key in hex
    print(my_private_key.hex())  # Print your private key in hex
    txout_scriptPubKey = P2PKH_scriptPubKey(my_address)
    response = send_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_index, txout_scriptPubKey)
    print(response.status_code, response.reason)
    print(response.text)  # Report the hash of transaction which is printed in this section result


In [ ]:
import requests

from bitcoin.core import b2x, lx, COIN, COutPoint, CMutableTxOut, CMutableTxIn, CMutableTransaction, Hash160
from bitcoin.core.script import *
from bitcoin.core.scripteval import VerifyScript, SCRIPT_VERIFY_P2SH


def send_from_custom_transaction(amount_to_send, txid_to_spend, utxo_index, txin_scriptPubKey, txin_scriptSig, txout_scriptPubKey):
    txout = create_txout(amount_to_send, txout_scriptPubKey)
    txin = create_txin(txid_to_spend, utxo_index)
    new_tx = create_signed_transaction(txin, txout, txin_scriptPubKey, txin_scriptSig)
    return broadcast_transaction(new_tx)


def create_txin(txid, utxo_index):
    return CMutableTxIn(COutPoint(lx(txid), utxo_index))


def create_txout(amount, scriptPubKey):
    return CMutableTxOut(amount*COIN, CScript(scriptPubKey))


def create_OP_CHECKSIG_signature(txin, txout, txin_scriptPubKey, seckey):
    tx = CMutableTransaction([txin], [txout])
    sighash = SignatureHash(CScript(txin_scriptPubKey), tx, 0, SIGHASH_ALL)
    sig = seckey.sign(sighash) + bytes([SIGHASH_ALL])
    return sig


def create_signed_transaction(txin, txout, txin_scriptPubKey, txin_scriptSig):
    tx = CMutableTransaction([txin], [txout])
    txin.scriptSig = CScript(txin_scriptSig)
    VerifyScript(txin.scriptSig, CScript(txin_scriptPubKey), tx, 0, (SCRIPT_VERIFY_P2SH,))
    return tx


def broadcast_transaction(tx):
    raw_transaction = b2x(tx.serialize())
    headers = {'content-type': 'application/x-www-form-urlencoded'}
    return requests.post(
        'https://api.blockcypher.com/v1/btc/test3/txs/push',
        headers=headers,
        data='{"tx": "%s"}' % raw_transaction,
    )


Now use the provided functions to make these transactions:

2.1: One input and two outputs transaction

2.2: Pay to MultiSig transaction (P2MS)


2.3: Custom Transaction (BitCoin Scripting)
